# Моделирование случайных величин и проверка гипотез о нормальном распределении
> ## Анализ нормального распределения

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.integrate import quad
from tabulate import tabulate

> ### Параметры нормального распределения

In [ ]:
mu = 0      # Математическое ожидание
sigma = 1   # Среднеквадратическое отклонение
size = 1000 # Размер выборки

> ### Генерация выборки

In [ ]:
np.random.seed(42)
sample = np.sort(np.random.normal(mu, sigma, size))

> ### Теоретические характеристики

In [ ]:
theoretical_mean = mu
theoretical_var = sigma**2

> ### Выборочные характеристики

In [ ]:
sample_mean = np.mean(sample)
sample_var = np.var(sample, ddof=0)  # Дисперсия с ddof=0 (N в знаменателе)
sample_var_unbiased = np.var(sample, ddof=1)  # Несмещенная дисперсия (N-1 в знаменателе)
sample_range = np.max(sample) - np.min(sample)
sample_median = np.median(sample)

> ### Вывод характеристик

In [ ]:
print("Теоретические характеристики:")
print(f"Математическое ожидание: {theoretical_mean:.4f}")
print(f"Дисперсия: {theoretical_var:.4f}\n")

print("Выборочные характеристики:")
print(f"Выборочное среднее: {sample_mean:.4f}")
print(f"Выборочная дисперсия (смещенная): {sample_var:.4f}")
print(f"Выборочная дисперсия (несмещенная): {sample_var_unbiased:.4f}")
print(f"Размах выборки: {sample_range:.4f}")
print(f"Выборочная медиана: {sample_median:.4f}")

> ## 2. Графики теоретической и выборочной функций распределения

> ### Теоретическая функция распределения

In [ ]:
x = np.linspace(mu - 4*sigma, mu + 4*sigma, 1000)
theoretical_cdf = norm.cdf(x, mu, sigma)

> ### Эмпирическая функция распределения

In [ ]:
def empirical_cdf(x, sample):
    return np.searchsorted(sample, x, side='right') / len(sample)

emp_cdf = np.vectorize(empirical_cdf, excluded=['sample'])(x, sample=sample)

> ### Построение графиков

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x, theoretical_cdf, 'b-', label='Теоретическая CDF', linewidth=2)
plt.step(x, emp_cdf, 'r--', label='Выборочная CDF', linewidth=1, where='post')
plt.title('Теоретическая и выборочная функции распределения')
plt.xlabel('x')
plt.ylabel('F(x)')
plt.legend()
plt.grid(True)
plt.show()

> ### Расхождение(Интегральная квадратичная разность)

In [ ]:
def divergence_measure(sample, theoretical_cdf_func, a, b):
    sample_sorted = np.sort(sample)
    ecdf = np.arange(1, len(sample_sorted)+1) / len(sample_sorted)
    tcdf = theoretical_cdf_func(sample_sorted)
    
    # Вычисляем интеграл как сумму на интервалах между точками выборки
    squared_diff = (ecdf - tcdf)**2
    integral = np.sum(squared_diff[:-1] * np.diff(sample_sorted))
    
    # Добавляем хвосты за пределами выборки
    integral += quad(lambda x: theoretical_cdf_func(x)**2, a, sample_sorted[0])[0]
    integral += quad(lambda x: (1 - theoretical_cdf_func(x))**2, sample_sorted[-1], b)[0]
    
    return integral

divergence = divergence_measure(sample, lambda x: norm.cdf(x, mu, sigma), mu-4*sigma, mu+4*sigma)
print(f"\nМера расхождения между теоретической и выборочной CDF: {divergence:.6f}")

> ## 3. Построение гистограммы и сравнение с теоретической плотностью

In [ ]:


plt.figure(figsize=(10, 6))
n, bins, patches = plt.hist(sample, bins=20, density=True, alpha=0.6, color='g', label='Нормированная гистограмма')


x_pdf = np.linspace(mu - 4*sigma, mu + 4*sigma, 1000)
theoretical_pdf = norm.pdf(x_pdf, mu, sigma)
plt.plot(x_pdf, theoretical_pdf, 'r-', linewidth=2, label='Теоретическая PDF')


bin_centers = 0.5 * (bins[:-1] + bins[1:])
theoretical_pdf_values = norm.pdf(bin_centers, mu, sigma)


max_diff = np.max(np.abs(n - theoretical_pdf_values))
print(f"\nМаксимум абсолютной разности между гистограммой и теоретической PDF: {max_diff:.6f}")

plt.title('Гистограмма выборки и теоретическая плотность распределения')
plt.xlabel('x')
plt.ylabel('Плотность')
plt.legend()
plt.grid(True)
plt.show()

> ## Таблица результатов

In [ ]:
table = []
for i in range(len(bin_centers)):
    table.append([
        f"{bins[i]:.2f} - {bins[i+1]:.2f}",
        f"{bin_centers[i]:.4f}",
        f"{n[i]:.4f}",
        f"{theoretical_pdf_values[i]:.4f}",
        f"{np.abs(n[i] - theoretical_pdf_values[i]):.4f}"
    ])


In [ ]:
headers = ["Интервал", "Середина интервала", "Высота гистограммы", 
           "Теоретическая плотность", "Абсолютная разность"]
print("\nТаблица сравнения гистограммы с теоретической плотностью:")
print(tabulate(table, headers=headers))

In [ ]:
print("\nВыводы:")
print("1. Выборочные характеристики близки к теоретическим для нормального распределения.")
print("2. Графики CDF показывают хорошее соответствие теоретического и эмпирического распределений.")
print(f"3. Мера расхождения между CDF составляет {divergence:.6f}.")
print(f"4. Максимальное расхождение между гистограммой и теоретической PDF: {max_diff:.6f}.")